In [1]:
from datasets import load_dataset

In [2]:
from babydragon.memory.indexes.memory_index import MemoryIndex


In [3]:
dataset_url = "Cohere/wikipedia-22-12-simple-embeddings"
dataset = load_dataset(dataset_url)['train'] 

Found cached dataset parquet (C:/Users/Tommaso/.cache/huggingface/datasets/Cohere___parquet/Cohere--wikipedia-22-12-simple-embeddings-94deea3d55a22093/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset.column_names

['id',
 'title',
 'text',
 'url',
 'wiki_id',
 'views',
 'paragraph_id',
 'langs',
 'emb']

In [5]:
values = dataset.select_columns(['text'])


In [6]:
values[0]['text']

'The 24-hour clock is a way of telling the time in which the day runs from midnight to midnight and is divided into 24 hours, numbered from 0 to 23. It does not use a.m. or p.m. This system is also referred to (only in the US and the English speaking parts of Canada) as military time or (only in the United Kingdom and now very rarely) as continental time. In some parts of the world, it is called railway time. Also, the international standard notation of time (ISO 8601) is based on this format.'

In [7]:
def concat_columns(example):
    column1='title'
    column2='text'
    example['merged_column'] = example[column1] + example[column2]
    return example

# Apply the concatenation function to the dataset
merged_docs = dataset.map(concat_columns)

Loading cached processed dataset at C:\Users\Tommaso\.cache\huggingface\datasets\Cohere___parquet\Cohere--wikipedia-22-12-simple-embeddings-94deea3d55a22093\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-eb8b46c02b8c3b22.arrow


In [8]:
merged_docs.column_names

['id',
 'title',
 'text',
 'url',
 'wiki_id',
 'views',
 'paragraph_id',
 'langs',
 'emb',
 'merged_column']

In [9]:
values = merged_docs.select_columns(['merged_column'])
embeddings = merged_docs.select_columns(['emb'])


In [10]:
del dataset

In [11]:
from babydragon.models.embedders.cohere import CohereEmbedder
cohere_embedder = CohereEmbedder()

In [12]:
cohere_embedder.get_embedding_size()

768

In [13]:
list_values = list(values['merged_column'][:5005])
list_embeddings = list(embeddings['emb'][:5005])

In [14]:
wiki_index = MemoryIndex(values=list_values,embeddings=list_embeddings, name='wikipedia_cohere_index', embedder=CohereEmbedder)

Creating a new index from a list of embeddings and values


In [15]:
wiki_index.faiss_query("What is the meaning of life?")

(['HinduismThe ultimate goal of life,according to Hinduism is "moksha", "nirvana" or "samadhi", but is understood in different ways in different schools.For example, Advaita Vedanta says that after attaining moksha a person knows their "soul, self" and identifies it as one with Brahman (Ultimate reality or cause of everything). The followers of Dvaita (dualistic) schools,state that after attaining moksha a person identify "soul, self" different from Brahman but very close to Brahman, and after attaining moksha one will spend eternity in a loka (higher planes). According to theistic schools of Hinduism, moksha is liberation from samsara, while for other schools such as the monistic school, moksha is possible in current life and is a psychological concept.',
  'JudaismJews see "mitzvot" as acts that sanctify – bring holiness – to the world and bring people and the world closer to God. Jews do the "mitzvot" to sanctify the physical world and the things in it, such as food and drink, cloth

In [16]:
from babydragon.chat.memory_chat import FifoChat


In [18]:
chatbot = FifoChat(model= "command", index_dict = {"wikiindex":wiki_index}, name="babyd_chatbot", max_index_memory = 2500, max_output_tokens= 400)

Index is available so using index prompts


In [19]:
chatbot.reply("What is the meaning of life?")

Trying to call Cohere API... using model: command


#### Question: 
 What is the meaning of life?

 #### Anwser: 
 
According to Hinduism, the meaning of life is to realize that one's soul is identical to the supreme soul, that the supreme soul is present in everything and everyone, all life is interconnected and there is oneness in all life.

"\nAccording to Hinduism, the meaning of life is to realize that one's soul is identical to the supreme soul, that the supreme soul is present in everything and everyone, all life is interconnected and there is oneness in all life."